In [2]:
import pandas as pd 
import re
import requests
from io import StringIO


def get_day_df(standort: str, date: str):
    """Lädt und parst Loggerdaten für ein gegebenes Datum & Standort."""

    def split_wr_column(series: pd.Series, wr: str) -> pd.DataFrame:
        """Spaltet WR-Spalte in beschriftete Teilspalten"""
        split_data = series.str.split(";", expand=True).astype(int)
        n = split_data.shape[1]
        if n < 2:
            raise ValueError(f"{wr}: Ungültige Anzahl an Werten ({n})")

        n_strings = (n - 2) // 2
        col_names = [f"{wr}_P"] \
                  + [f"{wr}_S{i+1}_P" for i in range(n_strings)] \
                  + [f"{wr}_sum"] \
                  + [f"{wr}_S{i+1}_Udc" for i in range(n_strings)]
        if (n - 2) % 2 ==1:
            col_names += [f"{wr}_T"]
        split_data.columns = col_names
        return split_data

    try:
        # URL zusammenbauen
        base_url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/"
        heute = datetime.now().strftime("%y%m%d")
        file_name = "min_day.js" if date == heute else f"min{date}.js"
        url = base_url + file_name

        # Request & Daten extrahieren
        response = requests.get(url)
        if response.status_code != 200:
            return f"Download fehlgeschlagen: {response.status_code}"

        matches = re.findall(r'="([^"]+)"', response.text)
        if not matches:
            return pd.DataFrame()

        raw_data = "\n".join(matches).replace("|", ",")
        df = pd.read_csv(StringIO(raw_data), sep=",", header=None)
        df[0] = pd.to_datetime(df[0], format="%d.%m.%y %H:%M:%S")
        df.columns = ["Datetime"] + [f"WR{i+1}" for i in range(len(df.columns) - 1)]

        # WR-Spalten aufspalten
        parts = [df[["Datetime"]]]
        for wr in df.columns[1:]:
            parts.append(split_wr_column(df[wr], wr))

        return pd.concat(parts, axis=1)

    except Exception as e:
        return f"Fehler beim Parsen: {e}"

In [3]:
import re
from datetime import datetime
import pandas as pd
import requests
import os

def _parse_date(ddmm_string):
    """Konvertiert 'dd.mm.yy' oder 'dd.mm.yyyy' -> 'YYYY-MM-DD' (ISO)"""
    s = ddmm_string.strip()
    try:
        if re.match(r'^\d{2}\.\d{2}\.\d{2}$', s):
            return datetime.strptime(s, '%d.%m.%y').date().isoformat()
        else:
            return datetime.strptime(s, '%d.%m.%Y').date().isoformat()
    except Exception:
        # falls unbekanntes Format: original zurückgeben
        return s

def parse_js_text(text, standort='Esslingen'):
    """
    Parst Text mit Zeilen wie:
    da[dx++]="13.06.23|4500;0|1905;0|1668;0"
    -> DataFrame long format mit Spalten: date, standort, wr, value
    wr wird als 'WR1','WR2',... bezeichnet.
    """
    rows = []
    # alle "..." Inhalte extrahieren (darin sind die records)
    matches = re.findall(r'"([^"]+)"', text)
    for m in matches:
        parts = m.split('|')
        if not parts:
            continue
        date_iso = _parse_date(parts[0])
        # alle folgenden tokens sind "value;0" - wir nehmen nur den value-Teil vor dem ';'
        for i, token in enumerate(parts[1:], start=1):
            token = token.strip()
            if token == '':
                continue
            val = token.split(';')[0]
            # versuche int, sonst float, sonst None
            try:
                val_num = int(val)
            except ValueError:
                try:
                    val_num = float(val.replace(',', '.'))
                except Exception:
                    val_num = None
            rows.append({
                'date': date_iso,
                'standort': standort,
                'wr': i,
                'value': val_num
            })
    df = pd.DataFrame(rows, columns=['date', 'standort', 'wr', 'value'])
    # date in datetime umwandeln (wenn möglich) und sortieren (neueste zuerst)
    try:
        df['date'] = pd.to_datetime(df['date'])
        df["date"] = df["date"].dt.date
        df = df.sort_values(['date', 'wr'], ascending=[False, True]).reset_index(drop=True)
    except Exception:
        pass
    return df


def update_all_days():
    standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof","karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]
    new_data_pull = []
    for s in standorte:
        url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{s}/visualisierung/days_hist.js"
        response = requests.get(url)
        new_data_pull.append(parse_js_text(response.text, standort=s))
        url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{s}/visualisierung/days.js"
        response = requests.get(url)
        new_data_pull.append(parse_js_text(response.text, standort=s))
    new_data_pull = pd.concat(new_data_pull) 

    if os.path.exists("days.parquet"):

        old_data = pd.read_parquet("days.parquet")
  
        final = pd.concat([new_data_pull,old_data]).drop_duplicates(subset=["date","standort","wr"],keep="first").sort_values(by="date")
        
        final.to_parquet("days.parquet",index=False)
    else:
        new_data_pull.drop_duplicates(subset=["date","standort","wr"]).sort_values(by="date").to_parquet("days.parquet",index=False)

update_all_days()


In [55]:
import polars as pl
from datetime import date

In [94]:

data_polars = pl.scan_parquet("days.parquet")
df_heute = (
    data_polars
    .filter(
        (pl.col("standort").str.to_lowercase() == "badboll") &
        (pl.col("date") == pl.lit(date.today()))
    )
    .collect(engine="streaming")
)
df_heute

date,standort,wr,value
date,str,i64,i64
2025-11-16,"""badboll""",1,0
2025-11-16,"""badboll""",2,10547
2025-11-16,"""badboll""",3,11217


In [97]:
data_polars = pl.scan_parquet("days.parquet")
df_heute = (
    data_polars
    .filter(
        (pl.col("standort").str.to_lowercase() == "badboll") &
        (pl.col("date") == pl.lit(date.today()))
    )
    .collect(engine="streaming")
)
df_heute

date,standort,wr,value
date,str,i64,i64
2025-11-16,"""badboll""",1,0
2025-11-16,"""badboll""",2,10547
2025-11-16,"""badboll""",3,11217


In [107]:
import polars as pl
from datetime import date
import calendar
import numpy as np

def get_ertrag_of_month(standort):
    data_polars = pl.scan_parquet("days.parquet")

    heute = date.today()
    akt_jahr = heute.year
    akt_monat = heute.month
    # Anzahl Tage im aktuellen Monat
    days_in_month = calendar.monthrange(akt_jahr, akt_monat)[1]
    
    df_monat = (
        data_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort) &
            (pl.col("date").dt.year() == akt_jahr) &
            (pl.col("date").dt.month() == akt_monat)
        )
        .group_by(["standort", "date"])
        .agg(
            pl.col("value").sum().alias("value_sum")/1000
        )
        .sort("date")   # optional: nach Datum sortieren
        .collect(engine="streaming")
    )

    numpy_array =  df_monat.to_pandas()["value_sum"].values
     # Array auffüllen bis zur Länge des Monats
    if len(numpy_array) < days_in_month:
        fill_length = days_in_month - len(numpy_array)
        numpy_array = np.pad(numpy_array, (0, fill_length), 'constant', constant_values=0)

    # Als Liste zurückgeben
    return np.round(numpy_array,1).tolist()
get_ertrag_of_month("badboll")

[23.4,
 8.8,
 54.0,
 74.1,
 67.7,
 74.4,
 57.2,
 33.7,
 8.4,
 38.5,
 51.7,
 51.4,
 53.4,
 20.5,
 26.6,
 21.8,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [109]:
def get_ertrag_of_year(standort):
    data_polars = pl.scan_parquet("days.parquet")

    heute = date.today()
    akt_jahr = heute.year
    
    # Daten für das aktuelle Jahr filtern
    df_jahr = (
        data_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("date").dt.year() == akt_jahr)
        )
        .with_columns([
            pl.col("date").dt.month().alias("month")  # Monat extrahieren
        ])
        .group_by("month")
        .agg(
            (pl.col("value").sum()/1000).alias("value_sum")  # kWh
        )
        .sort("month")
        .collect(engine="streaming")
    )

    # In ein dict für schnelles Auffüllen
    month_dict = {m: v for m, v in zip(df_jahr["month"], df_jahr["value_sum"])}
    
    # Array für 12 Monate erstellen
    ertrag_liste = [month_dict.get(i, 0) for i in range(1, 13)]
    
    # Runde auf 1 Nachkommastelle und als numpy Array zurückgeben
    return np.round(ertrag_liste, 1)

get_ertrag_of_year("badboll")

array([ 708.2, 1102.5, 2112.1, 2931.9, 3171.6, 3324.5, 2654.8, 2961.2,
       1785.4, 1284.9,  665.7,    0. ])

In [ ]:

data_polars = pl.scan_parquet("days.parquet")
standort="badboll"
heute = date.today()
akt_jahr = heute.year

# Daten für das aktuelle Jahr filtern
df_jahr = (
    data_polars
    .filter(
        (pl.col("standort").str.to_lowercase() == standort.lower()) &
        (pl.col("date").dt.year() == akt_jahr)
    )
    .with_columns([
        pl.col("date").dt.month().alias("month")  # Monat extrahieren
    ]).collect())

df_jahr

date,standort,wr,value,month
date,str,i64,i64,i8
2025-01-01,"""badboll""",2,16753,1
2025-01-01,"""badboll""",3,18869,1
2025-01-01,"""badboll""",1,0,1
2025-01-02,"""badboll""",1,0,1
2025-01-02,"""badboll""",3,2421,1
…,…,…,…,…
2025-11-15,"""badboll""",2,13083,11
2025-11-15,"""badboll""",1,0,11
2025-11-16,"""badboll""",1,0,11
